In [13]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Flatten, Conv2D
def build_model(hp):
    units1=hp.Int('units1',32,128, step=32)
    units2=hp.Int('units2',32,128, step=32)
    lr=hp.Choice('lr',[1e-2,1e-3,1e-4])

    in1=Input(shape=(10,), name="input1")
    x1=Dense(units1, activation='relu')(in1)
    x1=Dense(units2, activation='relu')(x1)

    in2=Input(shape=(64,64,3), name="input2")
    x2=Conv2D(32,(3,3), activation='relu')(in2)
    x2=Conv2D(64,(3,3), activation='relu')(x2)
    x2=Flatten()(x2)

    combined=concatenate([x1,x2])
    out1=Dense(1, name="output1")(combined)
    out2=Dense(10, activation='softmax', name="output2")(combined)
    model=Model(inputs=[in1,in2], outputs=[out1, out2])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=['mse','categorical_crossentropy'], metrics=['mae','accuracy'])
    return model
tuner=kt.RandomSearch(build_model, objective='val_loss', max_trials=10, executions_per_trial=2, directory='my_dir', project_name='multi_io_tuning')
x1,x2=np.random.rand(1000,10),np.random.rand(1000,64,64,3)
y1,y2=np.random.rand(1000,1),tf.keras.utils.to_categorical(np.random.randint(0,10,(1000,1)),10)
train, val= slice(800), slice(800,1000)
tuner.search([x1[train],x2[train]],[y1[train],y2[train]], validation_data=([x1[val],x2[val]],[y1[val],y2[val]]), epochs=5, batch_size=32)
best_model = tuner.get_best_models(1)[0]
best_model.fit([x1[train], x2[train]], [y1[train], y2[train]],
               epochs=5, batch_size=32)

Reloading Tuner from my_dir\multi_io_tuning\tuner0.json
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - loss: 2.5798 - output1_loss: 0.2317 - output1_mae: 0.3873 - output2_accuracy: 0.0883 - output2_loss: 2.3480
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - loss: 2.3033 - output1_loss: 0.0795 - output1_mae: 0.2433 - output2_accuracy: 0.1869 - output2_loss: 2.2237
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - loss: 1.9299 - output1_loss: 0.0594 - output1_mae: 0.2017 - output2_accuracy: 0.5001 - output2_loss: 1.8704
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - loss: 1.0387 - output1_loss: 0.0702 - output1_mae: 0.2033 - output2_accuracy: 0.9162 - output2_loss: 0.9685
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 153ms/step - loss: 0.2842 - output1_loss: 0.1148 - output1_mae: 0.2622 - output2_accuracy: 1.0000 - output2_loss: 0.1694
